# **Homework 2-1 Phoneme Classification**

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [1]:
#!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
#!unzip data.zip
#!ls 

## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np

print('Loading data ...')

data_root='/content/drive/MyDrive/timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


## Create Dataset

In [4]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [5]:
VAL_RATIO = 0.02

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (1205333, 429)
Size of validation set: (24599, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [6]:
BATCH_SIZE = 32

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [7]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

153

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [8]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer1 = nn.Linear(429, 4096)
        self.layer2 = nn.Linear(4096, 2048)
        self.layer3 = nn.Linear(2048, 1024)
        self.layer4 = nn.Linear(1024, 512)
        self.layer5 = nn.Linear(512, 256)
        self.out = nn.Linear(256, 39)

        """
        self.layer1 = nn.Linear(429, 1024)
        self.layer2 = nn.Linear(1024, 512)
        self.layer3 = nn.Linear(512, 128)
        self.out = nn.Linear(128, 39) 
        """

        self.act_fn = nn.ReLU()
        self.BN0 = nn.BatchNorm1d(429)
        self.BN1 = nn.BatchNorm1d(4096)
        self.BN2 = nn.BatchNorm1d(2048)
        self.BN3 = nn.BatchNorm1d(1024)
        self.BN4 = nn.BatchNorm1d(512)
        self.BN5 = nn.BatchNorm1d(256)
        self.drop_out1 = nn.Dropout(0.3)
        self.drop_out2 = nn.Dropout(0.5)

    def forward(self, x):
        x = self.BN0(x)

        x = self.layer1(x)
        x = self.BN1(x)
        x = self.act_fn(x)
        x = self.drop_out1(x)

        x = self.layer2(x)
        x = self.BN2(x)
        x = self.act_fn(x)
        x = self.drop_out1(x)

        x = self.layer3(x)
        x = self.BN3(x)
        x = self.act_fn(x)
        x = self.drop_out1(x)

        x = self.layer4(x)
        x = self.BN4(x)
        x = self.act_fn(x)
        x = self.drop_out1(x)

        x = self.layer5(x)
        x = self.BN5(x)
        x = self.act_fn(x)
        x = self.drop_out1(x)

        x = self.out(x)
        
        return x

## Training

In [9]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [10]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [11]:
# fix random seed for reproducibility
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 44               # number of training epoch
learning_rate = 0.0001       # learning rate

# the path where checkpoint saved
model_saved_path = "/content/drive/MyDrive/tmp_model/model.ckpt"
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0001, amsgrad=True)

# checkpoint mode
checkpoint = torch.load(model_saved_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']


DEVICE: cuda


In [12]:
# start training

best_acc = checkpoint['best_acc']
#best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                torch.save({
                              'epoch': epoch,
                              'model_state_dict': model.state_dict(),
                              'optimizer_state_dict': optimizer.state_dict(),
                              'train_loss': train_loss,
                              'train_acc': train_acc,
                              'val_loss': val_loss,
                              'val_acc': val_acc,
                              'best_acc': best_acc 
                            }, model_path)
                torch.save(model.state_dict(), model_saved_path)
                torch.save({
                              'epoch': epoch,
                              'model_state_dict': model.state_dict(),
                              'optimizer_state_dict': optimizer.state_dict(),
                              'train_loss': train_loss,
                              'train_acc': train_acc,
                              'val_loss': val_loss,
                              'val_acc': val_acc,
                              'best_acc': best_acc 
                            }, model_saved_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))

    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


[001/044] Train Acc: 0.746403 Loss: 0.770832 | Val Acc: 0.756494 loss: 0.741343
[002/044] Train Acc: 0.747317 Loss: 0.767950 | Val Acc: 0.752673 loss: 0.754431
[003/044] Train Acc: 0.747797 Loss: 0.766745 | Val Acc: 0.757185 loss: 0.739996
[004/044] Train Acc: 0.748023 Loss: 0.763898 | Val Acc: 0.756494 loss: 0.746592
[005/044] Train Acc: 0.748841 Loss: 0.761372 | Val Acc: 0.757510 loss: 0.746697
[006/044] Train Acc: 0.748854 Loss: 0.761067 | Val Acc: 0.763364 loss: 0.739674
saving model with acc 0.763
[007/044] Train Acc: 0.749863 Loss: 0.758781 | Val Acc: 0.760478 loss: 0.738608
[008/044] Train Acc: 0.750909 Loss: 0.755626 | Val Acc: 0.758202 loss: 0.742754
[009/044] Train Acc: 0.751686 Loss: 0.753604 | Val Acc: 0.760966 loss: 0.728274
[010/044] Train Acc: 0.751351 Loss: 0.752824 | Val Acc: 0.756535 loss: 0.740146
[011/044] Train Acc: 0.752949 Loss: 0.749797 | Val Acc: 0.758933 loss: 0.743050
[012/044] Train Acc: 0.752844 Loss: 0.749028 | Val Acc: 0.755315 loss: 0.751239
[013/044] Tr

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [13]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
checkpoint = torch.load(model_saved_path)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

Make prediction.

In [14]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [15]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))

In [17]:
%cp /content/prediction.csv /content/drive/MyDrive/output/prediction.c


# Testing log
---
# 1. Val = 0.2, Batch = 64, Sigmoid, Adam (lr = 0.0001, num_epoch = 20)

## [Upload: 0.68717]
##### [017/020] Train Acc: 0.779358 Loss: 0.674498 | Val Acc: 0.702492 loss: 0.943941, saving model with acc 0.702 

---
# 2. Val = 0.2, Batch = 32, ReLU, Adadelta (num_epoch = 20)
# (From 1: 減小B, 改ReLU, 改Adadelta)
#### [011/020] Train Acc: 0.425888 Loss: 2.169479 | Val Acc: 0.445255 loss: 2.100470, saving model with acc 0.445

---
# 3. Val = 0.1, Batch = 64, ReLU, Adam (lr = 0.0001, num_epoch = 20)
# (From 1: 改Val, 改ReLU)

## [Upload: 0.70362]

#### [007/020] Train Acc: 0.781556 Loss: 0.650488 | Val Acc: 0.719173 loss: 0.874977, saving model with acc 0.719

---
# 4. Val = 0.1, Batch = 64, ReLU, Adadelta (num_epoch = 20)

# (From 1: 改Val, 改ReLU, 改Adadelta)
#### [001/020] Train Acc: 0.586574 Loss: 1.439755 | Val Acc: 0.606330 loss: 1.429865, saving model with acc 0.606


---
# 5. Val = 0.1, Batch = 64, Sigmoid, Adam (lr = 0.0001, num_epoch = 20)

# (From 3: 改Sigmoid) 驗證是否是ReLU比較好
#### [015/020] Train Acc: 0.771501 Loss: 0.697592 | Val Acc: 0.707262 loss: 0.924157, saving model with acc 0.707


---
# 6. Val = 0.1, Batch = 64, ReLU, Nadam (lr = 2e-3, num_epoch = 20)
# (From 3: 改Nadam)
#### [019/020] Train Acc: 0.752201 Loss: 0.856589 | Val Acc: 0.688969 loss: 1.189685, saving model with acc 0.689

---
# 7. Val = 0.1, Batch = 64, ReLU, Nadam (lr = 0.0001, num_epoch = 20)
# (From 6: 改lr)

## [Upload: 0.70354]
#### [007/020] Train Acc: 0.781690 Loss: 0.650251 | Val Acc: 0.718043 loss: 0.880606, saving model with acc 0.718


---
# 8. Val = 0.1, Batch = 64, ReLU, Adagrad (lr = 0.01)
# (From 3: 改Adagrad, lr)

#### [009/020] Train Acc: 0.762591 Loss: 0.722741 | Val Acc: 0.717303 loss: 0.869923, saving model with acc 0.717


---
# 9. Val = 0.1, Batch = 64, ReLU, Adagrad (lr = 0.05, wd = 0.001)
# (From 8: 改lr, wd)

#### [017/020] Train Acc: 0.710140 Loss: 0.905975 | Val Acc: 0.700896 loss: 0.937002, saving model with acc 0.701

---
# 10. Val = 0.1, Batch = 64, ReLU, Adagrad (lr = 0.01, wd = 0.001)
# (From 8: 改wd)

## [Upload: 0.70433]
#### [019/020] Train Acc: 0.749886 Loss: 0.776568 | Val Acc: 0.719409 loss: 0.868873, saving model with acc 0.719

---
# 11. Val = 0.1, Batch = 64, ReLU, Adam (lr = 0.0001, wd = 0.001)
# (From 10: 改Adam)
## [Upload: 0.70810]

#### [017/020] Train Acc: 0.744398 Loss: 0.779707 | Val Acc: 0.723710 loss: 0.848050, saving model with acc 0.724

---
# 12. Val = 0.02, Batch = 64, ReLU, Adam (lr = 0.0001, wd = 0.001)
# (From 11: 改Val)

#### [020/020] Train Acc: 0.746101 Loss: 0.774290 | Val Acc: 0.729095 loss: 0.837620, saving model with acc 0.729

---
# 13. Val = 0.02, Batch = 64, ReLU, AdaW (lr = 0.0001, wd = 0.01)
# (From 12: 改AdamW, 參數預設)

#### [006/020] Train Acc: 0.767721 Loss: 0.695459 | Val Acc: 0.725924 loss: 0.843756, saving model with acc 0.726

---
# 14. Val = 0.02, Batch = 64, ReLU, Adagrad (lr = 0.0001, wd = 0.01)
# (From 12: 改Adagrad, 參數預設)

#### [018/020] Train Acc: 0.597101 Loss: 1.338628 | Val Acc: 0.615594 loss: 1.303488, saving model with acc 0.616


---
# 15. Val = 0.02, Batch = 64, ReLU, Adam (lr = 0.0001, wd = 0.001), Dropout = 0.2
# (From 12: 加入Dropout)

#### [020/020] Train Acc: 0.712875 Loss: 0.889066 | Val Acc: 0.728119 loss: 0.846611, saving model with acc 0.728

---
# 16. Val = 0.02, Batch = 64, ReLU, Adam (lr = 0.0001, wd = 0.001), Dropout = 0.1, 0.5, 0.1
# (From 15: 改Dropout)

#### [018/020] Train Acc: 0.679356 Loss: 1.012939 | Val Acc: 0.716208 loss: 0.903012, saving model with acc 0.716

---
# 17. Val = 0.02, Batch = 64, ReLU, Adam (lr = 0.0001, wd = 0.001),  batch norm
# (From 12: 加入batch norm)

#### [010/020] Train Acc: 0.758296 Loss: 0.735215 | Val Acc: 0.735314 loss: 0.820954, saving model with acc 0.735

---
# 18. Val = 0.02, Batch = 64, ReLU, Adam (lr = 0.0001, wd = 0.001),  batch norm, dropout = 0.5 (middile)
# (From 16: 加入batch norm, 改Dropout)

#### [015/020] Train Acc: 0.705983 Loss: 0.919125 | Val Acc: 0.731412 loss: 0.839901, saving model with acc 0.731

---
# 19. Val = 0.02, Batch = 64, ReLU, Adam (lr = 0.0001, wd = 0.001), Dropout = 0.5 (middile)
# (From 16: 改Dropout)

#### [018/020] Train Acc: 0.701185 Loss: 0.929748 | Val Acc: 0.719907 loss: 0.881123, saving model with acc 0.720

---
# 20. Val = 0.02, Batch = 64, ReLU, Adam (lr = 0.0001, wd = 0.001),  batch norm, epoch = 30
# (From 17: 改Epoch)
## [Upload: 0.71401]

#### [010/030] Train Acc: 0.758296 Loss: 0.735215 | Val Acc: 0.735314 loss: 0.820954, saving model with acc 0.735

---
# 21. Val = 0.02, Batch = 64, ReLU, Adam (lr = 0.0001, wd = 0.001), Dropout = 0.1 (all)
# (From 16: 改Dropout)

#### [020/020] Train Acc: 0.712875 Loss: 0.889066 | Val Acc: 0.728119 loss: 0.846611, saving model with acc 0.728

---
# 22. Val = 0.02, Batch = 64, ReLU, Adam (lr = 0.0001, wd = 0.001),  batch norm, dropout = 0.1 (all)
# (From 18: 改dropout)
## [Upload: 0.71920]

#### [015/020] Train Acc: 0.726784 Loss: 0.845392 | Val Acc: 0.742876 loss: 0.810006, saving model with acc 0.743

---
# 23. Val = 0.02, Batch = 64, ReLU, Adam (lr = 0.0001, wd = 0.001),  batch norm, dropout = 0.1, 0.5, 0.1
# (From 22: 改dropout)

#### [018/020] Train Acc: 0.688045 Loss: 0.988221 | Val Acc: 0.730152 loss: 0.853573, saving model with acc 0.730


---
# 24. Val = 0.02, Batch = 64, ReLU, Adam (lr = 0.0001, wd = 0.001),  batch norm (all), dropout = 0.1 (all), d: 1024, 512, 256, 128, l: 4
# (From 22: 加一層, dropout跟著layer)

#### [017/020] Train Acc: 0.725571 Loss: 0.851009 | Val Acc: 0.741209 loss: 0.807741, saving model with acc 0.741

---
# 25. Val = 0.02, Batch = 64, ReLU, Adam (lr = 0.0001, wd = 0.001),  batch norm (all), dropout = 0.1 (all), d: 2048, 1024, 512, 256, l: 4
# (From 24: 改dim)

#### [015/020] Train Acc: 0.738183 Loss: 0.801946 | Val Acc: 0.745193 loss: 0.792096, saving model with acc 0.745

---
# 26. Val = 0.02, Batch = 64, ReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.1 (all), d: 2048, 1024, 512, 256, l: 4
# (From 25: Adagrad)

## [Upload: 0.72730]

#### [014/020] Train Acc: 0.813472 Loss: 0.558870 | Val Acc: 0.751941 loss: 0.795578, saving model with acc 0.752

---

# 27. Val = 0.02, Batch = 64, ReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.1 (all), d: 2048, 1024, 512, 256, 128, l: 5
# (From 26: 增加layer)

### 跟26差不多（低一點點點）

---
# 28. Val = 0.02, Batch = 64, ReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.05 (all), d: 2048, 1024, 512, 256, l: 4
# (From 26: 減少dropout)

#### [006/020] Train Acc: 0.773432 Loss: 0.685245 | Val Acc: 0.749095 loss: 0.785736, saving model with acc 0.749

---
# 29. Val = 0.02, Batch = 32, ReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.1 (all), d: 2048, 1024, 512, 256, l: 4
# (From 26: 減少batchsize)

#### [008/020] Train Acc: 0.748473 Loss: 0.769647 | Val Acc: 0.750315 loss: 0.763709, saving model with acc 0.750

---
# 30. Val = 0.02, Batch = 16, ReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.1 (all), d: 2048, 1024, 512, 256, l: 4
# (From 29: 減少batchsize)

#### [013/020] Train Acc: 0.730069 Loss: 0.834682 | Val Acc: 0.750030 loss: 0.776581, saving model with acc 0.750

---
# 31. Val = 0.02, Batch = 128, ReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.1 (all), d: 2048, 1024, 512, 256, l: 4
# (From 30: 增加batchsize)

#### [007/020] Train Acc: 0.783077 Loss: 0.655175 | Val Acc: 0.748608 loss: 0.781434, saving model with acc 0.749

---
# 32. Val = 0.01, Batch = 64, ReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.1 (all), d: 2048, 1024, 512, 256, l: 4
# (From 26: 減小Val)


#### [009/020] Train Acc: 0.783233 Loss: 0.655085 | Val Acc: 0.729268 loss: 0.854641, saving model with acc 0.729

---
# 33. Val = 0.05, Batch = 64, ReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.1 (all), d: 2048, 1024, 512, 256, l: 4
# (From 26: 增加Val)

#### [008/020] Train Acc: 0.776095 Loss: 0.679058 | Val Acc: 0.744492 loss: 0.790004, saving model with acc 0.744

---
# 34. Val = 0.02, Batch = 64, ReLU, Adadelta (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.1 (all), d: 2048, 1024, 512, 256, l: 4
# (From 26: 換Adadelta)

##### 爛到不用記

---
# 35. Val = 0.02, Batch = 64, ReLU, Adadelta (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.1 (all), d: 4096, 2048, 1024, 512, 256, l: 5
# (From 26: 改layer, dimension)

#### [013/020] Train Acc: 0.811250 Loss: 0.567488 | Val Acc: 0.751169 loss: 0.796611, saving model with acc 0.751


---
# 36. Val = 0.02, Batch = 64, ReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.3 (all), d: 2048, 1024, 512, 256, l: 4
# (From 26: 改dropout)
## [Upload: 0.74022]

#### [020/020] Train Acc: 0.751780 Loss: 0.771806 | Val Acc: 0.756860 loss: 0.751556, saving model with acc 0.757


---
# 37. Val = 0.02, Batch = 64, ReLU6, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.1 (all), d: 2048, 1024, 512, 256, l: 4
# (From 26: 改ReLU6)

#### [014/020] Train Acc: 0.813472 Loss: 0.558870 | Val Acc: 0.751941 loss: 0.795578, saving model with acc 0.752

---
# 38. Val = 0.02, Batch = 64, RReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.1 (all), d: 2048, 1024, 512, 256, l: 4
# (From 36: 改RReLU)

#### [020/020] Train Acc: 0.744379 Loss: 0.780384 | Val Acc: 0.750030 loss: 0.766126, saving model with acc 0.750


---
# 39. Val = 0.02, Batch = 64, RReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.3 (all), d: 2048, 1024, 512, 256, l: 4
# (From 37: 改dropout)

#### [020/020] Train Acc: 0.744379 Loss: 0.780384 | Val Acc: 0.750030 loss: 0.766126, saving model with acc 0.750


---
# 40. Val = 0.02, Batch = 64, ReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.3 (all), d: 2048, 1024, 512, 256, l: 4
# (From 36: 改epoch = 40)
## [Upload: 0.74132]

#### [030/040] Train Acc: 0.766117 Loss: 0.724972 | Val Acc: 0.760315 loss: 0.750639, saving model with acc 0.760

---
# 41. Val = 0.02, Batch = 64, ReLU, Adagrad (lr = 0.01, wd = 0.0001),  batch norm (all), dropout = 0.3 (all), d: 2048, 1024, 512, 256, l: 4
# (From 36: 改wd)

#### [020/020] Train Acc: 0.744439 Loss: 0.775144 | Val Acc: 0.754218 loss: 0.748208, saving model with acc 0.754

---
# 42. Val = 0.02, Batch = 64, ReLU, Adagrad (lr = 0.01, wd = 0.01),  batch norm (all), dropout = 0.3 (all), d: 2048, 1024, 512, 256, l: 4
# (From 36: 改wd)

#### [018/020] Train Acc: 0.690835 Loss: 1.094645 | Val Acc: 0.738770 loss: 0.916497, saving model with acc 0.739

---
# 43. Val = 0.02, Batch = 64, ReLU, Adagrad (lr = 0.01, wd = 0.005),  batch norm (all), dropout = 0.3 (all), d: 2048, 1024, 512, 256, l: 4
# (From 36: 改wd)

#### [018/020] Train Acc: 0.716781 Loss: 0.950057 | Val Acc: 0.746331 loss: 0.829992, saving model with acc 0.746

---
# 44. Val = 0.02, Batch = 64, ReLU, Adagrad (lr = 0.01, wd = 0.0005),  batch norm (all), dropout = 0.3 (all), d: 2048, 1024, 512, 256, l: 4
# (From 36: 改wd)

#### [018/020] Train Acc: 0.751767 Loss: 0.761569 | Val Acc: 0.758527 loss: 0.744851, saving model with acc 0.759


---
# 45. Val = 0.02, Batch = 80, ReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.3 (all), d: 2048, 1024, 512, 256, l: 4
# (From 36: 改batch)

#### [020/020] Train Acc: 0.758614 Loss: 0.749284 | Val Acc: 0.759015 loss: 0.748557, saving model with acc 0.759


---
# 46. Val = 0.02, Batch = 64, ReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.3 (all), d: 4096, 2048, 1024, 512, l: 4
# (From 36: 改dimension)

#### [018/020] Train Acc: 0.782358 Loss: 0.660127 | Val Acc: 0.758689 loss: 0.752846, saving model with acc 0.759

---
# 47. Val = 0.02, Batch = 90, ReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.3 (all), d: 2048, 1024, 512, 256, l: 4
# (From 36: 改batch)

#### [020/020] Train Acc: 0.761686 Loss: 0.739953 | Val Acc: 0.758283 loss: 0.746602, saving model with acc 0.758


---
# 48. Val = 0.02, Batch = 100, ReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.3 (all), d: 2048, 1024, 512, 256, l: 4
# (From 36: 改batch)

#### [016/020] Train Acc: 0.754747 Loss: 0.761855 | Val Acc: 0.755600 loss: 0.757270, saving model with acc 0.756

---
# 49. Val = 0.02, Batch = 50, ReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.3 (all), d: 2048, 1024, 512, 256, l: 4
# (From 36: 改batch)

#### [017/020] Train Acc: 0.737868 Loss: 0.818511 | Val Acc: 0.754746 loss: 0.758974, saving model with acc 0.755


---
# 50. Val = 0.02, Batch = 70, ReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.3 (all), d: 2048, 1024, 512, 256, l: 4
# (From 36: 改batch)

#### [018/020] Train Acc: 0.750306 Loss: 0.776669 | Val Acc: 0.755884 loss: 0.773381, saving model with acc 0.756


---
# 51. Val = 0.02, Batch = 80, ReLU, Adagrad (lr = 0.01, wd = 0.0005),  batch norm (all), dropout = 0.3 (all), d: 4096, 2048, 1024, 512, l: 4
# (From 36: 改batch dimension wd)

#### [017/020] Train Acc: 0.807262 Loss: 0.568514 | Val Acc: 0.758771 loss: 0.759352, saving model with acc 0.759


---
# 52. Val = 0.02, Batch = 80, ReLU, Adagrad (lr = 0.01, wd = 0.0005),  batch norm (all), dropout = 0.5 (all), d: 4096, 2048, 1024, 512, l: 4
# (From 51: 改dropout)

#### [018/020] Train Acc: 0.736598 Loss: 0.814287 | Val Acc: 0.759177 loss: 0.735449, saving model with acc 0.759



---
# 53. Val = 0.02, Batch = 80, ReLU, Adam (lr = 0.01, wd = 0.0005),  batch norm (all), dropout = 0.3 (all), epoch = 40, d: 4096, 2048, 1024, 512, l: 4
# (From 51: 改Adam)

#### [004/040] Train Acc: 0.374957 Loss: 2.161849 | Val Acc: 0.439977 loss: 1.928192, saving model with acc 0.440


---
# 54. Val = 0.02, Batch = 80, ReLU, Adagrad (lr = 0.01, wd = 0.0005),  batch norm (all), dropout = 0.1 (all), d: 4096, 2048, 1024, 512, l: 4
# (From 51: 改dropout)

#### [004/020] Train Acc: 0.753637 Loss: 0.739950 | Val Acc: 0.747469 loss: 0.773155, saving model with acc 0.747


---
# 55. Val = 0.02, Batch = 32, ReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.3 (all), d: 2048, 1024, 512, 256, l: 4
# (From 36: 改epoch = 1000, es=60)
## [Upload: 0.74476]

#### [069/1000] Train Acc: 0.759293 Loss: 0.745836 | Val Acc: 0.765885 loss: 0.725001, saving model with acc 0.766

---
# 56. Val = 0.05, Batch = 32, ReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.3 (all), d: 2048, 1024, 512, 256, l: 4
# (From 55: Val改0.05)


---
# 57. Val = 0.02, Batch = 50, ReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.3 (all), d: 2048, 1024, 512, 256, l: 4
# (From 55: Batch改50)

#### [125/1000] Train Acc: 0.798755 Loss: 0.619928 | Val Acc: 0.766251 loss: 0.754857, saving model with acc 0.766

---
# 58. Val = 0.05, Batch = 50, ReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.3 (all), d: 2048, 1024, 512, 256, l: 4
# (From 56: Batch改50)

#### [046/1000] Train Acc: 0.771962 Loss: 0.706526 | Val Acc: 0.757679 loss: 0.760074, saving model with acc 0.758
---
# 59. Val = 0.02, Batch = 50, ReLU, Adagrad (lr = 0.01, wd = 0.001),  batch norm (all), dropout = 0.3 (all), d: 2048, 1024, 512, 256, 128, 64,l: 6
# (From 57: l改6)


